# Data Transform

In this notebook, we will ask you a series of questions to evaluate your findings from your EDA. Based on your response & justification, we will ask you to also apply a subsequent data transformation. 

If you state that you will not apply any data transformations for this step, you must **justify** as to why your dataset/machine-learning does not require the mentioned data preprocessing step.

The bonus step is completely optional, but if you provide a sufficient feature engineering step in this project we will add `1000` points to your Kahoot leaderboard score.

You will write out this transformed dataframe as a `.csv` file to your `data/` folder.

**Note**: Again, note that this dataset is quite large. If you find that some data operations take too long to complete on your machine, simply use the `sample()` method to transform a subset of your data.

### Feature enginnering:
* Inputting missing values
* Reshaping data
* Encoding
* Removing Values

In [1]:
import pandas as pd
import numpy as np

## Q1

Does your model contain any missing values or "non-predictive" columns? If so, which adjustments should you take to ensure that your model has good predictive capabilities? Apply your data transformations (if any) in the code-block below.

* No there are no missing values. All the columns except IsFlagggedFraud and IsFraud are predictive since I already removed the columns that identify bank information.

## Q2

Do certain transaction types consistently differ in amount or fraud likelihood? If so, how might you transform the type column to make this pattern usable by a machine learning model? Apply your data transformations (if any) in the code-block below.

* Transaction types like cash out and transder consistently have fraud behaivor.
* I can transform the type column by doing a one hot encoder since the transaction types are a categorical.

In [2]:

transactions = pd.read_csv("../data/bank_transactions.csv")
transactions = transactions.drop(columns=["nameOrig", "nameDest"])
transactions

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,PAYMENT,983.09,36730.24,35747.15,0.00,0.00,0,0
1,PAYMENT,55215.25,99414.00,44198.75,0.00,0.00,0,0
2,CASH_IN,220986.01,7773074.97,7994060.98,924031.48,703045.48,0,0
3,TRANSFER,2357394.75,0.00,0.00,4202580.45,6559975.19,0,0
4,CASH_OUT,67990.14,0.00,0.00,625317.04,693307.19,0,0
...,...,...,...,...,...,...,...,...
999995,PAYMENT,13606.07,114122.11,100516.04,0.00,0.00,0,0
999996,PAYMENT,9139.61,0.00,0.00,0.00,0.00,0,0
999997,CASH_OUT,153650.41,50677.00,0.00,0.00,380368.36,0,0
999998,CASH_OUT,163810.52,0.00,0.00,357850.15,521660.67,0,0


In [3]:
from sklearn.preprocessing import OneHotEncoder

# perform one-hot-encoding on a set of categorical columns

# TODO: select your choice of categorical columns
cat_features = transactions.select_dtypes(include=['object']).columns
cat_features

Index(['type'], dtype='object')

In [4]:
num_features = transactions.select_dtypes(include = ['int64', 'float64']).columns
num_features = num_features.drop(['isFraud','isFlaggedFraud'])
num_features

Index(['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest',
       'newbalanceDest'],
      dtype='object')

In [5]:
X_cat = transactions[cat_features]
X_num = transactions[num_features]

X_cat.head()

,type
0,PAYMENT
1,PAYMENT
2,CASH_IN
3,TRANSFER
4,CASH_OUT


In [6]:
ohe = OneHotEncoder()
X_cat_full = ohe.fit_transform(X_cat).toarray()

X_cat_full

array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.]], shape=(1000000, 5))

In [7]:
ohe.get_feature_names_out(cat_features)

array(['type_CASH_IN', 'type_CASH_OUT', 'type_DEBIT', 'type_PAYMENT',
       'type_TRANSFER'], dtype=object)

In [8]:
cat_names = ohe.get_feature_names_out(cat_features)

encoded_df = pd.DataFrame(X_cat_full, columns=cat_names, index=transactions.index)

encoded_df.head()

,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,0.0


In [9]:
X = pd.merge(X_num, encoded_df, left_index=True, right_index=True)

X

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,983.09,36730.24,35747.15,0.00,0.00,0.0,0.0,0.0,1.0,0.0
1,55215.25,99414.00,44198.75,0.00,0.00,0.0,0.0,0.0,1.0,0.0
2,220986.01,7773074.97,7994060.98,924031.48,703045.48,1.0,0.0,0.0,0.0,0.0
3,2357394.75,0.00,0.00,4202580.45,6559975.19,0.0,0.0,0.0,0.0,1.0
4,67990.14,0.00,0.00,625317.04,693307.19,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
999995,13606.07,114122.11,100516.04,0.00,0.00,0.0,0.0,0.0,1.0,0.0
999996,9139.61,0.00,0.00,0.00,0.00,0.0,0.0,0.0,1.0,0.0
999997,153650.41,50677.00,0.00,0.00,380368.36,0.0,1.0,0.0,0.0,0.0
999998,163810.52,0.00,0.00,357850.15,521660.67,0.0,1.0,0.0,0.0,0.0


In [10]:
# select the target
y = transactions["isFraud"]
y

0         0
1         0
2         0
3         0
4         0
         ..
999995    0
999996    0
999997    0
999998    0
999999    0
Name: isFraud, Length: 1000000, dtype: int64

## Q3

After exploring your data, you may have noticed that fraudulent transactions are rare compared to non-fraudulent ones. What challenges might this pose when training a machine learning model? What strategies could you use to ensure your model learns meaningful patterns from the minority class? Apply your data transformations (if any) in the code-block below.

Answer here

## Bonus (optional)

Are there interaction effects between variables (e.g., fraud and high amount and transaction type) that aren't captured directly in the dataset? Would it be helpful to manually engineer any new features that reflect these interactions? Apply your data transformations (if any) in the code-block below.

* adding a column that showed he difference between OldbalanceDest and NewbalanceDest
* Adding a variable to check of Balance was cleared because i saw that in my EDA (wasBalanceCleared = (newbalanceOrig == 0))
* destWasEmpty = (oldbalanceDest == 0)
* transferToEmpty = ((oldbalanceDest == 0) & (newbalanceDest > 0))

In [11]:
# write out newly transformed dataset to your folder
...